### Simplified Code for Image Recognition b/w Cat & Flower Images

In [89]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import matplotlib.pyplot as plt

In [90]:
from PIL import Image  # Add this import

In [91]:


# Define the SwinTransformer model (simplified)
class SwinTransformer(nn.Module):
    def __init__(self, num_classes=2):
        super(SwinTransformer, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=4, stride=4),
            nn.LayerNorm([96, 56, 56]),
            nn.GELU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.classifier = nn.Linear(96, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x



In [92]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])



In [93]:
# Load data
train_data = datasets.ImageFolder('C:/Users/sharm/Downloads/Cat_train', transform=transform)
test_data = datasets.ImageFolder('C:/Users/sharm/Downloads/Cat_test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Initialize model, loss, and optimizer
model = SwinTransformer().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.3)




In [94]:
# Training loop
def train(epochs):
    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')



In [95]:
# Test function
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Accuracy on test images: {100 * correct / total}%')



In [96]:
# Train the model
train(epochs=10)

# Test the model
test()



Epoch [1/10], Loss: 0.7219
Epoch [2/10], Loss: 0.7098
Epoch [3/10], Loss: 0.7033
Epoch [4/10], Loss: 0.6998
Epoch [5/10], Loss: 0.6981
Epoch [6/10], Loss: 0.6971
Epoch [7/10], Loss: 0.6962
Epoch [8/10], Loss: 0.6950
Epoch [9/10], Loss: 0.6936
Epoch [10/10], Loss: 0.6919
Accuracy on test images: 75.0%


In [97]:
# Function to predict on a single image
def predict_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    return train_data.classes[predicted.item()]

In [98]:
# Example usage
custom_image_path = "C:/Users/sharm/Downloads/Cat_test/class_2/s1.jpg"
prediction = predict_image(custom_image_path)
print(f"Prediction: {prediction}")

Prediction: class_2


In [99]:
from sklearn.metrics import precision_score, recall_score, f1_score

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Recall: 0.75
F1 Score: 0.7333333333333334


In the above code, the learning rate values influence the accuracy but they mostly fluctuate between 50-75.

3e-3 = 75% accuracy

3e-4 = 50% accuracy

1e-4 = 75% accuracy

1e-3 = 75% accuracy

1e-2 = 50% accuracy

2e-3 = 75% accuracy

2e-4 = 50% accuracy

5e-3 = 75% accuracy

5e-4 = 75% accuracy



### Key Metrics Overview:

Training Loss: The loss gradually decreases across the epochs, starting at 0.7219 in Epoch 1 and ending at 0.6919 in Epoch 10.

Test Accuracy: The accuracy on the test images reaches 75.0%, which indicates relatively good performance on the validation set.

### Key Observations:

Steady Loss Reduction: The training loss steadily decreases, which suggests that the model is learning consistently and making progress toward convergence. This smooth reduction indicates the optimization process is functioning well.

Higher Test Accuracy: The model achieves 75.0% accuracy on the test images, meaning it is correctly classifying three-quarters of the test samples. This is a significant improvement over random guessing and shows that the model has learned relevant patterns to distinguish between the two flower classes (Dandelion & Daisy).

### Positive Findings:

Effective Learning: The steady drop in training loss suggests that the model is not overfitting or underfitting and that its learning rate or other hyperparameters are likely well-tuned.

Good Generalization: With a test accuracy of 75.0%, the model seems to generalize well from the training data to unseen test data. This indicates that it has successfully captured meaningful features of the flower classes.

### Recommendations for Further Improvement:

Increase Epochs: Running the model for more epochs might help further reduce the training loss and potentially improve test accuracy.

Hyperparameter Tuning: Additional fine-tuning of hyperparameters (like learning rate, batch size, etc.) could help optimize the performance further.

Data Augmentation: While the model seems to generalize well, introducing more advanced data augmentation techniques might further improve accuracy, especially if there’s noise or variations in the test data.

Model Complexity: If more accuracy is required, increasing the model’s complexity (e.g., deeper layers, more attention heads in transformers) could be considered to better capture nuanced patterns in the flower dataset.

### Conclusion:

The model is performing well with a 75% test accuracy and consistently decreasing loss. The model could benefit from further training and tuning, but it is already showing signs of effective learning and generalization to the flower dataset.






